# Identyfikacja oddziaływań neutrin w detektorach ciekłoargonowych metodami uczenia maszynowego

## 1. Przygotowanie danych

Użycie GENIE do generacji zdarzeń zgodnie z instrukcją zawartą w pracy w rozdziale 3.1 powinno skutkować utworzeniem pliku o nazwie gntp.0.ghep.root. Plik ten należy zamienić na format o nazwie RooTracker komendą:

gntpc -i gntp.0.ghep.root -f rootracker

Z tak przygotowanego pliku można wyodrębnić najpotrzebniejsze dane do pliku .csv

In [1]:
import ROOT
import csv

Welcome to JupyROOT 6.26/10


In [2]:
nu_mu_file = ROOT.TFile("gntp.0.gtrac_mu.root")
nu_tau_file = ROOT.TFile("gntp.0.gtrac_tau.root")

In [3]:
# Lista kodów pdg cząstek o zerowym ładunku elektrycznym występujących w wygenerowanych zdarzeniach
neutral_particles = [-12, 12, -14, 14, -16, 16, -2112, 2112, -111, 111, 22, -3122, 3122, -3212, 3212,
                    -421, 421, -311, 311, -130, 130, 2000000101, 1000180400]

In [4]:
def extract_data(input_file, output_file, particles_list):
    tree = input_file.Get("gRooTracker") # Wgrywanie drzewa ROOT z danymi
    with open(output_file, 'w') as f:
        writer = csv.writer(f)
        for entry in tree:
            # Lista czteropędów każdej cząstki [px1, py1, pz1, E1, px2, py2, pz2, E2, ...]
            four_vectors = tree.StdHepP4
            row = [tree.EvtNum] # Numer oddziaływania
            # Iterowanie po wszystkich cząstkach w oddziaływaniu
            for particle in range(0, tree.StdHepN):
                particle_type = tree.StdHepPdg[particle] # Kod pdg cząstki
                # Wybieranie jedynie naładowanych końcowych produktów
                if tree.StdHepStatus[particle] == 1 and particle_type not in particles_list:
                    row.append(particle_type)
                    for counter in range(0, 4):
                        # Dodawanie do wiersza pędów i energi cząstki
                        row.append(four_vectors[4 * particle + counter])
            writer.writerow(row)

In [ ]:
extract_data(nu_mu_file, 'mu_data.csv', neutral_particles)
extract_data(nu_tau_file, 'tau_data.csv', neutral_particles)